# Homework assignment 3

### Student 1
- Name: Jerrijn Mandjes
- Student ID: 12573310

### Student 2
- Name: Thomas Komen
- Student ID: 12556963

## 1. Instructions

Fill in your name(s) and student ID(s) in the cell above. If you work alone, you may remove the part about "Student 2".

Then finish this notebook by writing an ASP encoding of a planning problem. Details about the planning problem and more detailed instructions for what to do can be found below.

## 2. The problem

In this homework assignment, you will write an ASP program to solve the
following (toy) planning problem.
This problem is about finding a way to deliver some packages from warehouses
to specified delivery locations, using several (electric) trucks.

The problem works in time steps. At each time step, each truck can perform exactly
one action—move to an adjacent node, wait, (un)load a package, and so on (more details below).
The problem operates on a map that consists of a graph with nodes and edges.
All nodes and edges are single capacity, meaning that only a single truck can be on a node or travel on an edge at each time step.
The task is to find a plan (a sequence of actions, one per truck per time step)
after which the goal state is achieved: all packages are delivered, the trucks are empty (i.e., without packages) and are located on a charging spot.

### The initial state

The initial state of the world consists of:
- a road network in the form of an undirected graph;
- the location (on the network) and initial load (number of available packages) of one or more warehouses;
- the location (on the network) and initial demand (number of packages to be delivered) of one or more delivery locations;
- the location (on the network) of one or more charging points; and
- the initial location, the initial load (how many packages aboard), the maximum load capacity, the initial battery level, and the maximum battery level of one or more trucks.

(These are all given as part of the input.)

### The actions (and their conditions and effects)

At each time step, each truck performs exactly one of the following actions:
- It can **wait**—the truck stays in the current location, the battery level stays the same.
- It can **move** to an adjacent node—the battery level goes down by 1 unit.
- If it is at a warehouse, the warehouse is not empty, and the truck still has capacity, it can **load** one single package from the warehouse into the truck—the battery level stays the same.
- If it is at a delivery location that still needs some packages, and the truck has at least one package, it can **unload** one single package at the delivery location—the battery level stays the same.
- If it is at a charging station, and the battery is not at full capacity, it can **charge**—the battery level increases by 1 unit.

The following additional restrictions have to be taken into account:
- If the battery level of a truck is 0, it cannot move to another node.
- At each location, there can be at most one truck at the same time.
- Trucks cannot pass each other on a piece of road—e.g., trucks at adjacent locations cannot swap places in a single time step.

### The goal

The goal is to achieve a state that has all of the following properties:
- All required deliveries must have been performed.
- Each truck has no packages aboard.
- Each truck is located at a charging station.

### The task

Finish this notebook by encoding this problem into ASP.
Make sure that your encoding works for any given input (that is of the right format).
That is, for each input, your code constructs an answer set program
whose answer sets correspond to the (shortest) sequences
of actions that achieve the goal from the initial state of the world—if the goal state can be reached within a given maximum number of steps.

## 3. Setting up
We begin with setting up and importing some relevant things.

In [ ]:
### DO NOT CHANGE THIS CELL

%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt

import clingo
from IPython.display import clear_output

## 4. Input
As input for our planning problem, we get a road network in the form of an undirected graph, specified as:
- a list of nodes
- a list of pairs of nodes, specifying the edges of the graph (these are to be interpreted as undirected)

Additionally, we get:
- a list of dictionaries specifying the factories (with their location on the graph and initial load)
- a list of dictionaries specifying the delivery points (with their location and initial demand)
- a list of dictionaries specifying the charging points (with their location)
- a list of dictionaries specifying the trucks, with:
  * their initial location
  * their initial load
  * their (load) capacity
  * their initial charge level
  * their maximum charge capacity

(See the example input below for the exact format of these dictionaries.)

Moreover, as part of the input, we get a maximum number of time steps, within which the goal should be reached.

### Example input

In [ ]:
### YOU MAY CHANGE THIS CELL TO TEST YOUR SOLUTION ON DIFFERENT INPUTS

nodes = list(range(1, 6+1))
edges = [
    (1,2),
    (2,3),
    (3,4),
    (4,5),
    (2,6),
    (3,6),
]
factories = [
    {
        'location': 1,
        'load': 10,
    },
]
delivery_points = [
    {
        'location': 4,
        'demand': 2,
    },
    {
        'location': 5,
        'demand': 2,
    },
]
charging_points = [
    {
        'location': 2,
    },
    {
        'location': 6,
    },
]
trucks = [
    {
        'location': 1,
        'load': 0,
        'capacity': 1,
        'charge level': 2,
        'charge capacity': 10,
    },
    {
        'location': 2,
        'load': 0,
        'capacity': 1,
        'charge level': 2,
        'charge capacity': 10,
    },
]

num_time_steps = 40

### Checking input assumptions

We will only consider inputs that have the following properties:
- no node is both a warehouse and a delivery location;
- no node is both a warehouse and a charging location; and
- no node is both a delivery and a charging location.

This means that your solution only needs to work for inputs that satisfy these assumptions.

The following code will check these assumptions, and throw an exception if they are not satisfied.

In [ ]:
### DO NOT CHANGE THIS CELL

warehouse_locations = [
    warehouse_info['location']
    for warehouse_info in factories
]
truck_locations = [
    truck_info['location']
    for truck_info in trucks
]
delivery_locations = [
    dp_info['location']
    for dp_info in delivery_points
]
charging_locations = [
    cp_info['location']
    for cp_info in charging_points
]

class InvalidInputError(Exception):
    pass

intersection = [location for location in warehouse_locations if location in delivery_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a warehouse and a delivery location')

intersection = [location for location in warehouse_locations if location in charging_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a warehouse and a charging location')

intersection = [location for location in delivery_locations if location in charging_locations]
if len(intersection) > 0:
    raise InvalidInputError(f'node {intersection[0]} is both a delivery and a charging location')

## 5. Assignment: writing an ASP program for finding solutions

Write down your encoding of the problem into ASP in cells in this section.
Make sure to interleave your code cells with appropriate text/markdown cells where you explain what you are doing.

In order to make the solution pretty-printing machinery in this notebook work, you should:
- use a predicate `do/3` to capture what actions the trucks take
  (where `do(T,Tr,A)` indicates that at time `T`, truck `Tr` takes action `A`); and
- use a predicate `truck_at/2` to capture the initial state of the trucks
  (where `truck_at(Tr,N)` indicates that the initial state of truck `Tr` is node `N`).

In [ ]:
### FILL IN YOUR SOLUTION HERE
### (AND ADD MORE CELLS DIRECTLY BELOW THIS ONE)

# Your program should go in the variable `asp_program`.

# The dummy below is intended to remind you to use do/3 and truck_at/2,
# to make the printing machinery work nicely.
asp_program = f"node(1..{len(nodes)}).\n"

for edge in edges:
    asp_program += f"edge({edge[0]}, {edge[1]}).\n"
asp_program += "edge(X,Y) :- edge(Y,X).\n"
for factory in factories:
    asp_program += f"factory(0, {factory['location']}, {factory['load']}).\n"
for dp in delivery_points:
    asp_program += f"delivery(0, {dp['location']}, {dp['demand']}).\n"
for cp in charging_points:
    asp_program += f"charging_point({cp['location']}).\n"
for i, truck in enumerate(trucks):
    asp_program += f"""truck_at(truck{i+1}, {truck['location']}).
    capacity(truck{i+1}, {truck['capacity']}).
    charge_level(0, truck{i+1}, {truck['charge level']}).
    charge_capacity(truck{i+1}, {truck['charge capacity']}).\n"""
asp_program += "truck_location(0, Truck, Location) :- truck_at(Truck, Location).\n"
asp_program += f"num_time_steps({num_time_steps}).\n"

asp_program += """
truck_load(0, Truck, 0) :- truck_at(Truck, _).
truck_location(0, Truck, Location) :- truck_at(Truck, Location).

action(move(X, Y)) :- edge(X, Y).
action(charge(Location)) :- charging_point(Location).
action(load(Location)) :- factory(_, Location, _).
action(unload(Location)) :- delivery(_, Location, _).
action(wait).


1 { do(Time+1, Truck,  A) : action(A)} 1:- truck_location(Time, Truck, _), num_time_steps(MaxTime), Time < MaxTime.
 

:- do(Time, Truck, move(X, Y)), not truck_location(Time-1, Truck, X).
:- do(Time, Truck, move(_, _)), charge_level(Time-1, Truck, 0).

:- do(Time, Truck, charge(Location)), not truck_location(Time-1, Truck, Location).
:- do(Time, Truck, charge(_)), charge_level(Time-1, Truck, Level), charge_capacity(Truck, MaxCharge), Level >= MaxCharge.

:- do(Time, Truck, load(Location)), not truck_location(Time-1, Truck, Location).
:- do(Time, Truck, load(_)), truck_load(Time-1, Truck, Load), capacity(Truck, MaxLoad), Load >= MaxLoad.
:- do(Time, Truck, load(Location)), factory(Time-1, Location, 0).

:- do(Time, Truck, unload(Location)), not truck_location(Time-1, Truck, Location).
:- do(Time, Truck, unload(_)), truck_load(Time-1, Truck, Load), Load = 0.
:- do(Time, Truck, unload(Location)), delivery(Time-1, Location, 0).

do(T+1, Truck, wait) :- do(T, Truck, wait), num_time_steps(MaxTime), T < MaxTime.

% Move actions
truck_location(Time, Truck, Location) :- do(Time, Truck, move(_, Location)).
charge_level(Time, Truck, Level-1) :- do(Time, Truck, move(_, _)), charge_level(Time-1, Truck, Level).
truck_load(Time, Truck, Load) :- do(Time, Truck, move(_, _)), truck_load(Time-1, Truck, Load).

%Load actions
truck_location(Time, Truck, Location) :- do(Time, Truck, load(Location)).
charge_level(Time, Truck, Level) :- do(Time, Truck, load(_)), charge_level(Time-1, Truck, Level).
truck_load(Time, Truck, Carry+1) :- do(Time, Truck, load(_)), truck_load(Time-1, Truck, Carry).
factory(Time, Location, Load-1) :- do(Time, Truck, load(Location)), factory(Time-1, Location, Load).

%Unload actions
truck_location(Time, Truck, Location) :- do(Time, Truck, unload(Location)).
charge_level(Time, Truck, Level) :- do(Time, Truck, unload(_)), charge_level(Time-1, Truck, Level).
truck_load(Time, Truck, Carry-1) :- do(Time, Truck, unload(_)), truck_load(Time-1, Truck, Carry).
delivery(Time, Location, Load-1) :- do(Time, Truck, unload(Location)), delivery(Time-1, Location, Load).

%Charge actions
truck_location(Time, Truck, Location) :- do(Time, Truck, charge(Location)).
charge_level(Time, Truck, Level+1) :- do(Time, Truck, charge(_)), charge_level(Time-1, Truck, Level).
truck_load(Time, Truck, Carry) :- do(Time, Truck, charge(_)), truck_load(Time-1, Truck, Carry).

%Wait actions
truck_location(Time, Truck, Location) :- do(Time, Truck, wait), truck_location(Time-1, Truck, Location).
charge_level(Time, Truck, Level-1) :- do(Time, Truck, wait), charge_level(Time-1, Truck, Level).
truck_load(Time, Truck, Load) :- do(Time, Truck, wait), truck_load(Time-1, Truck, Load).

%Maintaining existence of Factories and delivery points
factory(Time+1, Location, Load) :- factory(Time, Location, Load), not do(Time+1, _, load(Location)), num_time_steps(MaxTime), Time < MaxTime.
delivery(Time+1, Location, Loadd) :- delivery(Time, Location, Loadd), not do(Time+1, _, unload(Location)), num_time_steps(MaxTime), Time < MaxTime.
:- do(Time, Truck, Action), do(Time, Truck, Action2), Action != Action2.
:- truck_location(Time, Truck, Location), truck_location(Time, Truck2, Location), Truck != Truck2.

:- num_time_steps(MaxTime), truck_location(MaxTime, _, Location), not charging_point(Location).
:- num_time_steps(MaxTime), truck_load(MaxTime, _, Load), Load > 0.
:- delivery(MaxTime, Location, Demand), Demand > 0 , num_time_steps(MaxTime). 

#maximize { T : do(T,_,wait) }.

"""
asp_program += """
#show do/3.
#show truck_at/2.
"""
#
# Add when done: 
print(asp_program)

## 6. Finding and outputting solutions

The following cell contains some code that will be used to find (optimal) answer sets for your ASP program, to translate this to a solution for the planning problem, and to display this solution in a human-readable format.

You don't need to finish or change this cell in your solution.

In [ ]:
### DO NOT CHANGE THIS CELL

# Some machinery to (dynamically) print things in the notebook
log_str = ""
def reset_log():
    global log_str
    log_str = ""
def log(msg):
    global log_str
    print(msg, flush=True)
    log_str += f"{msg}\n"
def print_log():
    global log_str
    print(log_str, end='', flush=True)
    
def display_solution(model):
    """
    Displays a solution specified by the answer set
    in a human-readable format.
    
    Assumes that the answer set program uses do/3
    to specify actions, and uses truck_at/2 to
    specify the initial locations of the trucks.
    """
    actions = {}
    truck_locations = {}
    for atom in model.symbols(atoms=True):
        if atom.name == "do":
            time = atom.arguments[0].number
            truck = str(atom.arguments[1])
            action = str(atom.arguments[2])
            if time not in actions:
                actions[time] = {}
            actions[time][truck] = action
        elif atom.name == "truck_at":
            truck_locations[str(atom.arguments[0])] = str(atom.arguments[1])
    solution_str = "\n"
    solution_str += " "*5 + "".join([
        f"{truck:<5} @ {truck_locations[truck]:<10}"
        for truck in sorted(truck_locations)
    ]) + "\n"
    solution_str += "-"*(5+18*len(truck_locations)) + "\n"
    for time in sorted(actions):
        time_str = f"{time}:"
        solution_str += f"{time_str:<4} " + "".join([
            f"{actions[time][truck]:<18}"
            for truck in sorted(truck_locations)
        ]) + "\n"
    return solution_str

def find_and_output_solution(
    asp_program,
    timeout=10,
    num_threads=1,
):
    """
    Calls the ASP solver to find answer sets,
    and translates these to solutions,
    which are printed in human-readable format.
    """

    reset_log()
    log("..Grounding..")
    
    control = clingo.Control([f"--parallel-mode={num_threads}"])
    control.add("base", [], asp_program)
    control.ground([("base", [])])

    control.configuration.solve.models = 1
    control.configuration.solve.opt_mode = "optN"
    
    def interpret_model(model):
        clear_output(wait=True)
        print_log()
        print(display_solution(model))

    if timeout > 0:
        log(f"..Solving with timeout {timeout}s..")
        with control.solve(
            async_=True,
            on_model=lambda model: interpret_model(model),
        ) as handle:
            finished = handle.wait(timeout)
            if not finished:
                log("..Stopped after timeout..")
            else:
                log("..Stopped..")
                time = control.statistics['summary']['times']['total']
                log(f"..Total solving time: {time:.2f} seconds..")
    else:
        log("..Solving..")
        control.solve(
            on_model=lambda model: interpret_model(model),
        )
        log("..Stopped..")
        time = control.statistics['summary']['times']['total']
        log(f"..Total solving time: {time:.2f} seconds..")

## 7. For your convenience: visualizing (part of) the input

For your (and our) convenience, let's visualize the input to the problem. The road network will be displayed as a graph. Nodes with a warehouse are blue. Nodes with a delivery location are green. Nodes with a charging point are purple. All other nodes are red. Nodes where there is a truck initially are labelled with "T".

Note that this doesn't display all the information about the input. For example, the truck (maximum) battery level and (maximum) load is not displayed, and neither is the load/demand of the warehouses and delivery locations.

In [ ]:
### NO NEED TO CHANGE THIS CELL
### (UNLESS THE IMAGE DOESN'T SHOW WELL FOR SOME INPUT YOU ARE USING)

plt.rcParams["figure.autolayout"] = True
plt.rcParams["figure.figsize"] = (8, 6)
graph = nx.Graph()

ax = plt.gca()
ax.margins(0.08)

options = {
    "edgecolors": "tab:gray",
    "node_size": 2000,
    "font_color": "white",
    "font_size": 14,
}
graph.add_nodes_from(nodes)
graph.add_edges_from(edges)
layout = nx.spring_layout(graph, seed=42)

labels = {
    node:
    (f"{node} T" if node in truck_locations else f"{node}")
    for node in nodes
}

def node_color(node):
    if node in warehouse_locations:
        return "tab:blue"
    elif node in delivery_locations:
        return "tab:green"
    elif node in charging_locations:
        return "tab:purple"
    else:
        return "tab:red"
colors = [
    node_color(node)
    for node in nodes
]

nx.draw(
    graph,
    layout,
    ax=ax,
    node_color=colors,
    labels=labels,
    with_labels=True,
    **options
)

plt.show()

## 8. Finding and representing solutions

Finally, let's call the solver and display the solution that it might find.

Note that it will display a solution when it finds one, even if it then keeps searching for a better solution. Subsequent better (shorter) solutions will then be displayed, always displaying only the current best solution. Finally, after the solving stopped (possibly due to a time out), it will display "..Stopped.." below the best found solution.

In [ ]:
### NO NEED TO CHANGE THIS CELL
### (UNLESS YOU WANT TO SET A SOLVING TIMEOUT
###  OR USE A DIFFERENT NUMBER OF PARALLEL THREADS)

find_and_output_solution(
    asp_program,
    timeout=0,
    num_threads=4,
)